# Part 1

In [2]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]

In [8]:
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [11]:
from bs4 import BeautifulSoup
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipage= requests.get(url).text

# Parse html/xml codes from webpage.
soup = BeautifulSoup(wikipage,'xml')

# get the table in wikipage
table = soup.find('table')

listPostcode = []
listBorough = []
listNeighbourhood = []

for row in table.find_all('tr'):      
    cells = row.find_all('td')
    if cells:   # if cells has elements
        Postcode_var = cells[0].find(text = True)
        Borough_var = cells[1].find(text = True)
        Neighbourhood_var = cells[2].find(text = True).strip()    # .strip() to remove new line character '\n' at end of text.
    else:
        continue    # skip to next row if no elements
    

    # skip to next row if Borough = 'Not assigned'    
    if Borough_var == 'Not assigned': 
        continue
    
    # if Neighbourhood = 'Not assigned', then Neighborhood will be the same as Borough   
    if Neighbourhood_var == 'Not assigned': 
        Neighbourhood_var = Borough_var

    
    listPostcode.append(Postcode_var)
    listBorough.append(Borough_var)
    listNeighbourhood.append(Neighbourhood_var)
listUniqPostcode = set(listPostcode)
print(f'Number of unique Postcode: {len(listUniqPostcode)}')
print(f'Number of all Postcode: {len(listPostcode)}')
listNewPostcode = []
listNewBorough = []
listNewNeighbourhood = []


for postcode in listUniqPostcode:
    p_var = ''; b_var = ''; n_var = ''; 
    for idx, item in enumerate(listPostcode):
        if item == postcode:
            p_var = item;
            b_var = listBorough[idx]
            if n_var == '':    # if Neighbourhood hasn't got value due to new postcode
                n_var = listNeighbourhood[idx]
            else:     # if Neighbourhood already has value due to same postcode
                n_var = n_var + ', ' + listNeighbourhood[idx]
                
    listNewPostcode.append(p_var)
    listNewBorough.append(b_var)
    listNewNeighbourhood.append(n_var)

dict = {'Postcode':listNewPostcode, 'Borough':listNewBorough, 'Neighbourhood':listNewNeighbourhood}
df = pd.DataFrame.from_dict(dict)

Number of unique Postcode: 103
Number of all Postcode: 210


In [12]:
df

,Postcode,Borough,Neighbourhood
0,M4H,East York,Thorncliffe Park
1,M1W,Scarborough,L'Amoreaux West
2,M1N,Scarborough,"Birch Cliff, Cliffside West"
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
...,...,...,...
98,M4K,East Toronto,"The Danforth West, Riverdale"
99,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
100,M1J,Scarborough,Scarborough Village
101,M9L,North York,Humber Summit


# Part 2

In [14]:
url = 'https://cocl.us/Geospatial_data'
dflatlong = pd.read_csv(url, index_col = None)
dfnew = pd.merge(df, dflatlong, left_on = "Postcode", right_on = "Postal Code")
dfnew.drop('Postal Code', axis = 1, inplace = True)
dfnew.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4H,East York,Thorncliffe Park,43.705369,-79.349372
1,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389
2,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711


# Part 3

In [15]:
import folium
# Create folium map
torontoLat, torontoLong = 43.6532, -79.3832
map = folium.Map(location = [torontoLat, torontoLong], zoom_start = 10)


# Create circle markers and add to map
for lat, long, borough, neighborhood in zip(dfnew['Latitude'], dfnew['Longitude'], dfnew['Borough'], dfnew['Neighbourhood']):
    label = neighborhood + ', ' + borough
    label = folium.Popup(label, parse_html = True)
    marker = folium.CircleMarker(
            [lat, long],
            radius = 5,
            popup = label,
            color = 'blue',
            fill_color = 'Red',
            fill_opacity = 0.7,
            line_opacity = 0.2)
    
    marker.add_to(map)  
    

map

In [16]:
CLIENT_ID = 'QCBQABOL0LMDBKDOYIE2RDAUMJUVQDNHU3VIWQ1EWDW3WFRF'
CLIENT_SECRET = 'DXFDFTTNAN04LMZ4LYKILF3XJFAYI3N2AS2YBOSTHILBEKP3'
VERSION = '20180605'
dfscar = dfnew[dfnew['Borough'] == 'Scarborough'].reset_index(drop = True)
dfscar.head(7)
dfscar.shape

(17, 5)

In [17]:


scaraddress = 'Scarborough,Toronto'
scarlat = 43.773077
scarlong = -79.257774

# Create folium map
scarmap = folium.Map(location = [scarlat, scarlong], zoom_start = 11)

# Create circle markers and add to map
for lat, long, neighbourhood in zip(dfscar['Latitude'], dfscar['Longitude'], dfscar['Neighbourhood']):
    label = folium.Popup(neighbourhood, parse_html = True)
    marker = folium.CircleMarker(
            [lat, long],
            radius = 5,
            popup = label,
            color = 'blue',
            fill_color='Green',
            fill_opacity = 0.7,
            line_opacity = 0.2)
    
    marker.add_to(scarmap)  
    
LIMIT = 50
radius = 500

def exploreVenues(names, lats, longs, radius = 500):
    
    listvenues = []
    for name, lat, long in zip(names, lats, longs):
        print(name)
            
        # create url of API request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # create request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # extract desired info of each nearby venue
        listvenues.append([(
            name, 
            lat, 
            long, 
            result['venue']['name'], 
            result['venue']['location']['lat'], 
            result['venue']['location']['lng'],  
            result['venue']['categories'][0]['name']) for result in results])

    dfnearbyvenues = pd.DataFrame([item for listvenue in listvenues for item in listvenue])
    dfnearbyvenues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(dfnearbyvenues)

dfscarvenues = exploreVenues(names = dfscar['Neighbourhood'],
                                   lats = dfscar['Latitude'],
                                   longs = dfscar['Longitude']
                                  )
dfscarOnehot = pd.get_dummies(dfscarvenues[['Venue Category']], prefix = "", prefix_sep = "")

# add column Neighborhood into one-hot dataframe at first column
dfscarOnehot['Neighborhood'] = dfscarvenues['Neighborhood'] 
cols = [dfscarOnehot.columns[-1]] + list(dfscarOnehot.columns[:-1])
dfscarOnehot = dfscarOnehot[cols]


dfscarGroup = dfscarOnehot.groupby('Neighborhood').mean().reset_index()

def topFrequentVenues(row, topNum):
    row_categories = row.iloc[1:]
    row_categories = row_categories.sort_values(ascending = False)   # sort descending
    
    return row_categories.index.values[0:topNum]

topNum = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top frequent venues
cols = ['Neighborhood']
for ind in np.arange(topNum):
    try:
        cols.append('{}{} Most Frequent Venue'.format(ind+1, indicators[ind]))
    except:
        cols.append('{}th Most Frequent Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = cols)
neighborhoods_venues_sorted['Neighborhood'] = dfscarGroup['Neighborhood']

for ind in np.arange(dfscarGroup.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = topFrequentVenues(dfscarGroup.iloc[ind, :], topNum)

neighborhoods_venues_sorted

L'Amoreaux West
Birch Cliff, Cliffside West
Guildwood, Morningside, West Hill
Upper Rouge
Agincourt North, L'Amoreaux East, Milliken, Steeles East
East Birchmount Park, Ionview, Kennedy Park
Cliffcrest, Cliffside, Scarborough Village West
Maryvale, Wexford
Rouge, Malvern
Dorset Park, Scarborough Town Centre, Wexford Heights
Woburn
Cedarbrae
Agincourt
Highland Creek, Rouge Hill, Port Union
Clarks Corners, Sullivan, Tam O'Shanter
Clairlea, Golden Mile, Oakridge
Scarborough Village


,Neighborhood,1st Most Frequent Venue,2nd Most Frequent Venue,3rd Most Frequent Venue,4th Most Frequent Venue,5th Most Frequent Venue,6th Most Frequent Venue,7th Most Frequent Venue,8th Most Frequent Venue,9th Most Frequent Venue,10th Most Frequent Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Coffee Shop,Playground,Park,Vietnamese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Cosmetics Shop,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Intersection,Park,Bus Line,Ice Cream Shop,Metro Station,Soccer Field,Fast Food Restaurant,Construction & Landscaping,Fried Chicken Joint
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Intersection,Chinese Restaurant,Gas Station,Noodle House,Italian Restaurant,Pharmacy,Convenience Store,Shopping Mall,Bank
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Furniture / Home Store,Pet Store,Chinese Restaurant,College Stadium,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Hobby Shop,Bus Station,Department Store,Coffee Shop,Bank,Bar,General Entertainment,Gas Station,Furniture / Home Store
9,"Guildwood, Morningside, West Hill",Rental Car Location,Medical Center,Spa,Bank,Electronics Store,Mexican Restaurant,Breakfast Spot,Intersection,Department Store,Convenience Store


In [18]:
from sklearn.cluster import KMeans

dfscarCluster = dfscarGroup.drop('Neighborhood', 1)
k = 5   # number of clusters

# create and train model
model = KMeans(n_clusters = k, random_state = 0)
model.fit(dfscarCluster)

model.labels_[0:10]
len(model.labels_)

dfscarfinal = dfscar
dfscarfinal = dfscarfinal.drop(16)
len(dfscarfinal)

dfscarfinal['Cluster Labels'] = model.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dfscarfinal = dfscarfinal.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

dfscarfinal

import matplotlib.cm as cm
import matplotlib.colors as colors

# create folium map
mapclusters = folium.Map(location = [scarlat, scarlong], zoom_start = 11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
arrcolors = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in arrcolors]

# Create circle markers and add to map
markers_colors = []
for lat, long, neighbourhood, cluster in zip(dfscarfinal['Latitude'], dfscarfinal['Longitude'], dfscarfinal['Neighbourhood'], dfscarfinal['Cluster Labels']):
    label = folium.Popup(str(neighbourhood) + ' Cluster ' + str(cluster), parse_html = True)
    marker = folium.CircleMarker(
            [lat, long],
            radius = 5,
            popup = label,
            color = rainbow[cluster-1],            
            fill_color = rainbow[cluster-1],
            fill_opacity=0.7,
            line_opacity = 0.2)
    
    marker.add_to(mapclusters)
       
mapclusters